In [63]:
import pandas as pd
import re

In [51]:
def payer_category(x):
    payers = {
        'Gross Charge': 'gross',
        'Discounted Cash Price': 'cash',
        'Minimum Negotiated Charge': 'min',
        'Maximum Negotiated Charge': 'max'
    }

    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [52]:
file = 'pricing-transparency-charge.csv'
df = pd.read_csv(file)

In [53]:
cols = df.columns.tolist()
drop_cols = [col for col in cols if 'Unnamed' in col]
df.drop(drop_cols, axis=1, inplace=True)
print(df.columns.tolist())

['CDM Item Number', 'Revenue Code', 'Service ID', 'Service Mod1', 'Service Mod2', 'Service Description', ' Gross Charge ', ' Discounted Cash Price ', ' Minimum Negotiated Charge ', ' Maximum Negotiated Charge ', ' Payer Negotiated Charge: AARP (Plan: All) ', ' Payer Negotiated Charge: AARP- MedicareComplete UnitedHealthCare (Plan: All) ', ' Payer Negotiated Charge: Adminastar Blue Cross of Kentucky (Plan: All) ', ' Payer Negotiated Charge: Aetna (Plan: All) ', ' Payer Negotiated Charge: Aetna Better Health (LA) (Plan: All) ', ' Payer Negotiated Charge: Aetna Medicare Advantage (Plan: All) ', ' Payer Negotiated Charge: Blue Advantage of LA (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of LA (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of LA MCR Adv (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of TX (Plan: All) ', ' Payer Negotiated Charge: CHC Louisiana (Plan: All) ', ' Payer Negotiated Charge: Cigna (Plan: All) ', ' Payer Negot

In [54]:
df.rename(columns={
    'CDM Item Number': 'local_code',
    'Revenue Code': 'rev_code',
    'Service ID': 'code',
    'Service Description': 'description',

}, inplace=True)

In [56]:
cols = df.columns.tolist()
id_vars = cols[:6]
value_vars = cols[6:]
df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [57]:
df['modifier'] = df['Service Mod1'] + " " + df['Service Mod2']
df.drop(columns=['Service Mod1', 'Service Mod2'], inplace=True)

In [58]:
df['payer'] = df['payer'].str.strip()

In [64]:
df['hcpcs_cpt'] = df['code'].str[:5]
df.loc[df['code'].str.len() == 7, 'modifier'] = df['code'].str[-2:]

TypeError: expected string or bytes-like object

In [60]:
df['payer_category'] = df['payer'].apply(payer_category)

In [ ]:
df['hospital_id'] = '190118'

In [62]:
df.to_csv('desoto.csv', index=False)